In [5]:
# Initial imports
import os
import requests
import json
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi

In [33]:
#hardcoded variables to be replaced by user input and environment variable calls in the future
tickers = ['MSFT','GOOG']
start_date = '2020-01-01'
end_date = '2020-12-31'
timeframe = '1D'
alpaca_api_key = 'PKURMS5ELMEHV4ZGBXTQ'
alpaca_secret_key = 'NhYNWSU4szskB7qUMl2FXPqYoQZXwL50ahf54jds'

In [35]:
# Create the Alpaca API object
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2")
# Get current price data for tickers
df_portfolio = alpaca.get_bars(
    tickers,
    timeframe,
    start = start_date,
    end = end_date
).df
#Create a dataframe of closing prices


NameError: name 'df_closing_prices' is not defined

,close,high,low,trade_count,open,volume,vwap,symbol
timestamp,,,,,,,,
2020-01-02 05:00:00+00:00,1367.37,1368.14,1341.5500,41395,1341.55,1555727,1358.592568,GOOG
2020-01-03 05:00:00+00:00,1360.66,1372.50,1345.5436,28616,1347.86,1324192,1363.116060,GOOG
2020-01-06 05:00:00+00:00,1394.21,1396.50,1350.0000,43019,1350.00,1851121,1387.386204,GOOG
2020-01-07 05:00:00+00:00,1393.34,1402.99,1390.3800,37868,1397.94,1616127,1396.288195,GOOG
2020-01-08 05:00:00+00:00,1404.32,1411.58,1390.8400,38570,1392.08,1663718,1403.612932,GOOG
